In [ ]:
import pandas as pd

splits = {'train': 'train.csv', 'validation': 'valid.csv', 'test': 'test.csv'}
df = pd.read_csv("hf://datasets/chengxuphd/liar2/" + splits["train"])

In [ ]:
df.head()

,id,label,statement,date,subject,speaker,speaker_description,state_info,true_counts,mostly_true_counts,half_true_counts,mostly_false_counts,false_counts,pants_on_fire_counts,context,justification
0,13847,5,"90 percent of Americans ""support universal bac...","October 2, 2017",government regulation;polls and public opinion...,chris abele,"Chris Abele is Milwaukee County Executive, a p...",wisconsin,1,4,5,3,5,2,a tweet,"""Universal"" is the term for background checks ..."
1,13411,1,Last year was one of the deadliest years ever ...,"May 19, 2017",after the fact;congress;criminal justice;histo...,thom tillis,Thom Tillis is a Republican who serves as U.S....,north carolina,0,2,7,3,2,0,a press release supporting the Back The Blue A...,"Sen. Thom Tillis, a North Carolina Republican,..."
2,10882,0,"Bernie Sanders's plan is ""to raise your taxes ...","October 28, 2015",taxes,chris christie,"Chris Christie announced June 6, 2023 that he ...",national,21,20,27,11,17,8,"Boulder, Colo","Christie said that Sanders’s plan is ""to raise..."
3,20697,4,Voter ID is supported by an overwhelming major...,"December 8, 2021",voter id laws,lee zeldin,Lee Zeldin is a Republican representing New Yo...,new york,1,2,0,0,0,0,a Tweet,Zeldin claimed voter identification requiremen...
4,6095,2,"Says Barack Obama ""robbed Medicare (of) $716 b...","August 12, 2012",federal budget;history;medicare;retirement,mitt romney,Mitt Romney is a U.S. senator from Utah. He ra...,national,31,33,58,35,32,19,"an interview on ""60 Minutes""","Romney said, ""There's only one president that ..."


In [ ]:
df.describe()

,id,label,true_counts,mostly_true_counts,half_true_counts,mostly_false_counts,false_counts,pants_on_fire_counts
count,18369.000000,18369.000000,18369.000000,18369.000000,18369.000000,18369.000000,18369.000000,18369.000000
mean,11707.543198,2.246230,12.140345,20.436333,29.735859,46.493113,221.033861,101.415102
std,6721.341931,1.589746,22.330635,33.312096,44.053042,78.560404,437.411321,187.831286
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5895.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
50%,11730.000000,2.000000,3.000000,4.000000,6.000000,6.000000,6.000000,2.000000
75%,17512.000000,4.000000,17.000000,24.000000,35.000000,53.000000,233.000000,140.000000
max,23372.000000,5.000000,124.000000,165.000000,163.000000,245.000000,1410.000000,570.000000


In [ ]:
df.columns

Index(['id', 'label', 'statement', 'date', 'subject', 'speaker',
       'speaker_description', 'state_info', 'true_counts',
       'mostly_true_counts', 'half_true_counts', 'mostly_false_counts',
       'false_counts', 'pants_on_fire_counts', 'context', 'justification'],
      dtype='object')

In [ ]:
import pandas as pd
from transformers import BertTokenizer

# Dataset split paths
splits = {
    'train': 'train.csv',
    'validation': 'valid.csv',
    'test': 'test.csv'
}

# Load datasets from Hugging Face Datasets hub
train_df = pd.read_csv("hf://datasets/chengxuphd/liar2/" + splits["train"])
val_df   = pd.read_csv("hf://datasets/chengxuphd/liar2/" + splits["validation"])
test_df  = pd.read_csv("hf://datasets/chengxuphd/liar2/" + splits["test"])

# Combine text columns
def combine_text(row):
    statement = row['statement']
    context = row['context'] if pd.notna(row['context']) else ''
    justification = row['justification'] if pd.notna(row['justification']) else ''
    return f"{statement} {context} {justification}"

# Binary label mapping: 0 = Fake, 1 = Real
label_map = {1: 0, 2: 0, 3: 0, 4: 1, 5: 1}

def prepare_dataset(df):
    df = df.copy()
    df['label_mapped'] = df['label'].map(label_map)
    df = df.dropna(subset=['label_mapped'])  # Drop rows with unmapped labels (NaN)
    texts = df.apply(combine_text, axis=1).tolist()
    labels = df['label_mapped'].astype(int).tolist()
    return texts, labels

train_texts, train_labels = prepare_dataset(train_df)
val_texts, val_labels     = prepare_dataset(val_df)
test_texts, test_labels   = prepare_dataset(test_df)

# Tokenize
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings   = tokenizer(val_texts, truncation=True, padding=True, max_length=128)
test_encodings  = tokenizer(test_texts, truncation=True, padding=True, max_length=128)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
import torch

class LIARDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)  # <--- important
        return item

    def __len__(self):
        return len(self.labels)



train_dataset = LIARDataset(train_encodings, train_labels)
val_dataset   = LIARDataset(val_encodings, val_labels)
test_dataset  = LIARDataset(test_encodings, test_labels)


In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir='./logs',
    logging_steps=10
)


In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids
    return {
        'accuracy': accuracy_score(labels, preds),
        'f1_macro': f1_score(labels, preds, average='macro'),
        'f1_weighted': f1_score(labels, preds, average='weighted')
    }


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)


<ipython-input-18-183e30d7aaa8>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 1.2428109645843506, 'eval_accuracy': 0.8224674022066198, 'eval_f1_macro': 0.7911641596046451, 'eval_f1_weighted': 0.8232783670408679, 'eval_runtime': 14.8471, 'eval_samples_per_second': 134.302, 'eval_steps_per_second': 8.419, 'epoch': 5.0}


In [ ]:
!unzip -q "/content/fine_tuned_model.zip"

In [ ]:

save_path = "/content/fine_tuned_model"

trainer.save_model(save_path)
tokenizer.save_pretrained(save_path)


('/content/fine_tuned_model/tokenizer_config.json',
 '/content/fine_tuned_model/special_tokens_map.json',
 '/content/fine_tuned_model/vocab.txt',
 '/content/fine_tuned_model/added_tokens.json')

In [ ]:
text = "NASA found aliens on the moon"  # Your test claim
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)


In [ ]:
import torch

# Move model to appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Tokenize and move inputs to the same device
inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)

# Predict
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

# Interpret result
# Assuming predicted_class is one of [0, 1, 2, 3, 4]
predicted_class = torch.argmax(logits, dim=1).item()

# Map 5-class prediction to binary label
five_to_binary_map = {
    0: 0,  # pants-fire → fake
    1: 0,  # false      → fake
    2: 0,  # barely-true→ fake
    3: 1,  # half-true  → real
    4: 1   # true       → real
}

binary_label = five_to_binary_map[predicted_class]
label_text = "Real" if binary_label == 1 else "Fake"

print("Prediction:", label_text)



Prediction: Real


In [ ]:
!pip install serpapi

In [ ]:
!pip install google-search-results

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32010 sha256=331368b55c96547fcd15bc5b69495444dbac8d9f0e7aedeedf950ef2e1bba4d0
  Stored in directory: /root/.cache/pip/wheels/6e/42/3e/aeb691b02cb7175ec70e2da04b5658d4739d2b41e5f73cd06f
Successfully built google-search-results


In [ ]:
claim = "NASA found aliens on the moon"

from serpapi.google_search import GoogleSearch

def google_news_search(query):
    params = {
        "engine": "google",
        "q": query,
        "api_key": "084ae1811171136e03abfa744a17014599e2771e1b81817a38ce1ab56456b7f4",
        "gl": "in",
        "tbm": "nws"
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    return [article["title"] + " - " + article["link"] for article in results.get("news_results", [])]

results = google_news_search(claim)
# for r in results:
#     #print(r)

from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')
claim_embedding = model.encode(claim)

max_sim = 0
for article in results:
    article_embedding = model.encode(article)
    sim = util.cos_sim(claim_embedding, article_embedding).item()
    #print(f"{sim:.2f} - {article}")
    max_sim = max(max_sim, sim)


if max_sim > 0.7:
    print("✅ This claim is supported by a trusted source.")
else:
    print("⚠️ This claim might be fake or unverified.")


⚠️ This claim might be fake or unverified.
